## SCRAPPY
- Using Langchain build an Agentic AI
- Scrapes top e-commerce websites for desired products

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.tools import Tool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.tools import StructuredTool

from pydantic import BaseModel
import ast # For structured output (Convert string representation of list to Python list)

# from langchain.prompts import PromptTemplate
# from langchain.chains.llm import LLMChain
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains import ConversationalRetrievalChain # this allows query with memory

from dotenv import load_dotenv
import os


In [13]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY_2")

In [14]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.1)


In [15]:
from utility.outputFormat import FinalResult # this is a custom schema for saving product listings
from utility.save import SaveResult

- ```IMPORT CUSTOM TOOLS HERE AS FUNCTIONS```

In [ ]:
from scrappers.olx_scrapper import OlxScrapper
from scrappers.daraz import DarazScrapper

- ```Setup of the Scrapper Agent```

In [17]:
systemPrompt = """You are a helpful agent that scrapes listings from various platforms based on user queries. 
You will use the tools provided to gather information and return it in a structured format without any additional text.

User Query: {query}"""

generic_scraping_description = """A tool to scrape {name} for listings based on a query. 
It returns a list of dictionary with keys 'name', 'URL', 'price', 'rating', 'description'."""

save_scraping_description = """Saves the scraped listings to a local CSV file. Expects a list of dictionaries along with a file name."""

- `Scrapper Tools`

In [18]:

OlxScrapperTool = Tool(
    name="olx_scrapper",
    func=OlxScrapper,
    description=generic_scraping_description.format(name="OLX"),
)

DarazScrapperTool = Tool(
    name="daraz_scrapper",
    func=DarazScrapper,
    description=generic_scraping_description.format(name="Daraz")
)

- `Utility Tools`

In [19]:
# SaveScrapingTool = StructuredTool.from_function(
#     name="save_scraping",
#     func=SaveResult,
#     description=save_scraping_description,
#     # args_schema=SaveResultParams
# )

In [20]:
tools = load_tools(["llm-math"], llm=llm)

tools.extend([OlxScrapperTool, DarazScrapperTool]) # as we add more scrappers, we can extend this list
# tools.extend([SaveScrapingTool]) # as we add more utility tools, we can extend this list

- ```Function and Usage of Scrapper Agent```

In [21]:
def ScrapeListings(query: str) -> FinalResult:
    """
    Scrape listings from various platforms based on the query.
    
    Args:
        query (str): The search query to use for scraping.
        
    Returns:
        list: A list of dictionaries containing the scraped data.
    """
    
    query = systemPrompt.format(query=query)
    
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    # print(type(agent.agent))
    
    response = agent.invoke(query)
    return ast.literal_eval(response['output']) # Convert the string representation of the list back to a Python list

## NOTE  
- Right now search is only done based on the product, it does not support any sort of filters whatsoever 

In [ ]:
SaveResult(ScrapeListings("Get prices for XBOX ONE X from daraz"), filename="daraz_laptops.csv")



> Entering new AgentExecutor chain...
```json
{
  "action": "daraz_scrapper",
  "action_input": "XBOX ONE X"
}
```
Observation: [{'name': 'XBOX ONE X', 'price': '1000', 'URL': 'daraz.pk/search=laptop', 'rating': '4/5', 'description': "CPU: i5, GPU: RTX 1060 TI, 8 GB DDR4 RAM, 11'' LCD"}]
Thought:Action:
```json
{
  "action": "Final Answer",
  "action_input": "[{'name': 'XBOX ONE X', 'price': '1000', 'URL': 'daraz.pk/search=laptop', 'rating': '4/5', 'description': \"CPU: i5, GPU: RTX 1060 TI, 8 GB DDR4 RAM, 11'' LCD\"}]"
}
```

> Finished chain.
[{'name': 'XBOX ONE X', 'price': '1000', 'URL': 'daraz.pk/search=laptop', 'rating': '4/5', 'description': "CPU: i5, GPU: RTX 1060 TI, 8 GB DDR4 RAM, 11'' LCD"}]


'Items saved successfully to daraz_laptops.csv'